In [101]:
# General
import pandas as pd
import numpy as np
import datetime
import os
import glob
import sqlite3
from math import ceil

# Data viz
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

#pre-processing
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer

#feature selection
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE,f_classif
from sklearn.linear_model import LogisticRegression


import warnings
warnings.filterwarnings('ignore')

# Importing data

In [52]:
df = pd.read_csv('../data/MACIMIDE_Global_Expatriate_Dual_Citizenship_Dataset_V5.00.csv')

try:
    wdi = pd.read_csv('../data/WDIData.csv',usecols = range(65))
except:
    #saves the WDIData table without extracting the zip file
    with zipfile.ZipFile('../data/WDI_csv.zip') as myzip:
        wdi = pd.read_csv(myzip.open('WDIData.csv'),usecols = range(65))
        country = pd.read_csv(myzip.open('WDICountry.csv'))
        ind = pd.read_csv(myzip.open('WDISeries.csv'))

# Cleaning data

1. Keeping only the countries in WDI data
2. Keep years from 2000 to 2020

In [55]:
# 1. Keep only countries
cname = wdi.drop_duplicates(subset='Country Name')['Country Name']
wdi = wdi.loc[~wdi['Country Name'].isin(cname[:49])]

# 2. Keep only years we want
df = df.query('Year >= 2000 & Year <2020')
wdi = pd.concat([wdi.iloc[:,:4],wdi.iloc[:,43:-1]],axis=1)

# Changing world Region and selecting some countries

In [7]:
country_old = df.loc[df.world_region.isna()]['country'].unique()
df = df.loc[~df['country'].isin(country_old)]
def world_region(x):
    if x == 1:
        return 'Africa'
    elif x == 2:
        return 'Asia'
    elif x == 3:
        return 'Europe'
    elif x== 6:
        return 'Oceania'
    else:
        return 'America'    
    
df['world_region'] = df['world_region'].apply(lambda x:world_region(x) )

In [8]:
America = ["United States of America",'Canada','Mexico','Brazil', 'Colombia','Chile','Argentina']
Europe = ['Italy', 'Germany', 'Denmark', 'Poland', 'United Kingdom (of Great Britain and Northern Ireland)','France','Netherlands']
Extreme_Orient = ['China',  'Thailand', 'Australia', 'India',  'Azerbaijan','Japan']
Africa = ['South Africa', 'Djibouti', 'Morocco', 'Nigeria', 'Botswana']
Countries = [c for i in (America,Europe,Africa,Extreme_Orient) for c in i]

df_country = df.copy()
df_country = df.loc[df['country'].isin((Countries))]
len(df_country['country'].unique())

25

# Missing Values Treatment

First we are going to filter indicators with more than 20% of the data as nan values along countries and years. Inputation in thoses cases would change too much the analysis

In [57]:
Indicator_names = np.mean(1 -wdi.groupby('Indicator Name').count().iloc[:,3:-1]/217,axis =1)
Indicator_little_nan = pd.DataFrame(Indicator_names[Indicator_names<.20].sort_values(),columns=['% of nan'])
Indicator_little_nan.head(50)

,% of nan
Indicator Name,
Adjusted savings: mineral depletion (current US$),0.000000
Population growth (annual %),0.001613
"Population, total",0.001613
Population density (people per sq. km of land area),0.003226
Land area (sq. km),0.003226
Surface area (sq. km),0.005991
Urban population,0.010829
Urban population growth (annual %),0.010829
Rural population (% of total population),0.010829


In [60]:
# select indicators with less nan values
filtered_wdi = wdi.loc[wdi['Indicator Name'].isin(Indicator_little_nan.index)]

Now, for the 396 indicator left we should try to inputate the missing values by some of the below methods:
- KNNImputer
- mode (by Population density and GDP per capta)
- mean (by Population density and GDP per capta)

the *'by Population density and GDP per capta'* suppose to find the best mode and mean along countries, otherwise inputation would be too bias. Both indicatiors have few nan values and take in consideration important factors in selecting similar countries.

In [63]:
# knn imputer
imputer = KNNImputer(n_neighbors=5)
filtered_wdi.iloc[:,4:] = imputer.fit_transform(filtered_wdi.iloc[:,4:])
filtered_wdi

array([[ 21.38799645,  25.29571562,  20.29628758, ...,  97.7       ,
         98.71562195,  97.7       ],
       [  7.9210607 ,   7.46696885,   7.26975704, ...,  97.09197324,
         98.30960335,  96.90219005],
       [ 59.00685156,  64.76212622,  65.27360246, ...,  99.5       ,
         99.90214539, 100.        ],
       ...,
       [ 54.31999969,  54.02999878,  52.54000092, ...,  42.75      ,
         43.02000046,  42.36000061],
       [ 39.34999847,  38.72999954,  38.45000076, ...,  32.81000137,
         32.65000153,  31.25      ],
       [ 70.        ,  70.        ,  70.        , ...,  86.875     ,
         86.875     ,  86.875     ]])

In [ ]:
# mode

# mean

# Pre processing Data


In [70]:
#2019 data
wdi_pivoted = filtered_wdi.pivot(index='Country Code',columns='Indicator Name', values='2019')

dual_city_groups_2019 = df.loc[df.Year==2019].set_index('ISO3')[['Dualcit_grouped']]

wdi_scaled = wdi_pivoted.join(dual_city_groups_2019).dropna(subset=['Dualcit_grouped'])


scaler = MinMaxScaler()

wdi_scaled.iloc[:,:-1] = scaler.fit_transform(wdi_scaled.iloc[:,:-1])

# Selecting Features

### Filter Methods
- Kendalls correlation
- Anova correlation

### Wrapper methods
- RFE

### Assamble Methods
- random forest split

In [77]:
cor_kendall = wdi_scaled.corr(method ='kendall')
cor_kendall[['Dualcit_grouped']].sort_values('Dualcit_grouped',ascending=False)

,Dualcit_grouped
Dualcit_grouped,1.000000
"Unemployment, youth female (% of female labor force ages 15-24) (modeled ILO estimate)",0.188937
"Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)",0.180462
"Compulsory education, duration (years)",0.177421
"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)",0.172459
...,...
"Manufacturing, value added (constant LCU)",-0.170043
Adjusted savings: energy depletion (current US$),-0.170637
Merchandise imports from economies in the Arab World (% of total merchandise imports),-0.178047
Methane emissions in energy sector (thousand metric tons of CO2 equivalent),-0.179264


In [100]:
model = LogisticRegression()
rfe = RFE(estimator = model, n_features_to_select = 2)
X_rfe = rfe.fit_transform(X = wdi_scaled.iloc[:,:-1], y = wdi_scaled[['Dualcit_grouped']])

selected_features = pd.Series(rfe.support_, index = wdi_scaled.iloc[:,:-1].columns)
rank = pd.Series(rfe.ranking_, index = wdi_scaled.iloc[:,:-1].columns)

features_rfe = pd.concat([selected_features,rank],axis=1)
features_rfe.sort_values(1).head(20)

,0,1
"Agriculture, forestry, and fishing, value added (annual % growth)",True,1
"Population, total",True,1
Adjusted savings: particulate emission damage (current US$),False,2
"Foreign direct investment, net inflows (% of GDP)",False,3
Forest area (% of land area),False,4
Fixed telephone subscriptions (per 100 people),False,5
"Secondary education, duration (years)",False,6
Access to electricity (% of population),False,7
Mobile cellular subscriptions (per 100 people),False,8
Rural population,False,9


In [115]:
f_st = pd.Series(f_classif(wdi_scaled.iloc[:,:-1], wdi_scaled[['Dualcit_grouped']])[0],index=wdi_scaled.iloc[:,:-1].columns,name='f_statistic')
p_val = pd.Series(f_classif(wdi_scaled.iloc[:,:-1], wdi_scaled[['Dualcit_grouped']])[1],index=wdi_scaled.iloc[:,:-1].columns,name='p_value')

pd.concat([f_st,p_val],axis=1).sort_values('p_value')

,f_statistic,p_value
"Exports of goods, services and primary income (BoP, current US$)",5.107599,0.006908
Urban population growth (annual %),5.067053,0.007179
Population growth (annual %),5.067053,0.007179
"Exports of goods and services (BoP, current US$)",5.034199,0.007407
"Imports of goods, services and primary income (BoP, current US$)",4.921402,0.008245
...,...,...
External health expenditure (% of current health expenditure),0.177926,0.837143
Households and NPISHs final consumption expenditure (% of GDP),0.174362,0.840127
"Services, value added (current LCU)",0.170579,0.843305
Surface area (sq. km),0.146447,0.863869


In [98]:
X_train, X_val, y_train, y_val = train_test_split(wdi_scaled.iloc[:,:-1],wdi_scaled[['Dualcit_grouped']], test_size = 0.3, random_state = 0, stratify = wdi_scaled[['Dualcit_grouped']], shuffle = True)

#no of features
nof_list=np.arange(1,50)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    model = LogisticRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_val_rfe = rfe.transform(X_val)
    model.fit(X_train,y_train)
    
    score = model.score(X_val,y_val)
    score_list.append(score)
    
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
        
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

Optimum number of features: 1
Score with 1 features: 0.655172


# Merging the tables

In [238]:
def merge_tables(df,indicator_name):
    
    wdi_indicator = wdi.loc[wdi['Indicator Name']==indicator_name].set_index('Country Code').T.iloc[3:-1,:].reset_index()
    wdi_indicator['index'] = wdi_indicator['index'].astype(int)
    wdi_indicator = wdi_indicator.loc[wdi_indicator['index']>=2000]
    
    year = np.array([])
    ISOCODE = np.array([])
    value =  np.array([])
    
    for ISO in wdi_indicator.columns[1:-1]:
        ISOCODE_ = np.full((len(wdi_indicator),),ISO)
        ISOCODE = np.append(ISOCODE,ISOCODE_)
        
        year = np.append(year,wdi_indicator['index'].values)
        value = np.append(value,wdi_indicator[ISO].values)

        
    ISOCODE_SER = pd.Series(ISOCODE)
    year_ser = pd.Series(year)
    value = pd.Series(value).astype(float)
    final_wdi = pd.DataFrame([ISOCODE_SER,year_ser,value]).T.rename(columns={0:'ISO3',1:'Year',2:indicator_name})
    final_wdi['Year'] = final_wdi['Year'].astype(int)
    df_merge = df.merge(final_wdi, right_on=['ISO3','Year'],left_on = ['ISO3','Year'] )
    return   df_merge
    

In [239]:
#create a list of indicators names to merge with the dual cit base

df_merge = df.copy()
for ind in indi_to_use:
    df_merge = merge_tables(df_merge,str(ind))